### Recommender

In [ ]:
def get_association_rules():
   # returns data-frame with columns antecedents and consequents 
   print("To Do")
   

def recommender(product_id, n=5):
   
   # find frequent items
   association_rules = get_association_rules()
   
   # get the antecedents
   antecedents = association_rules["antecedents"]
   
   # check if our product has association rule
   if product_id in antecedents:
      # get the consequents of the product_id products 
      consequents = association_rules[association_rules['antecedents'] == product_id]
      
      # get top 5 products for each consequents:
      recommendations = DataFrame() # )
      for consequent in consequents:
         similar_items = get_similar_items(consequent, 10) # this function exists, change it so it takes the dataframe as well
         recommendations = pd.concat([recommendations, similar_items], ignore_index=True)
         
      recommendations = recommendations.sort_values(by='rrf_score', ascending=False).reset_index(drop=True)
      recommendations = recommendations[:n]
      
      return recommendations
   
   else:
      # in case it does not exist find the most similar antecedents to this one 
      
   
         
      
   
   

   
